In [1]:
from tqdm import tqdm
from utils.dataset import AI4VN_AirDataLoader
from utils.evaluation import eval_regression_model

air_data_loader = AI4VN_AirDataLoader()

/home/thutt36/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Scikit-learn ensemble models

In [2]:
from utils.models import EnsembleModel

### Train models

In [3]:
X_train, X_test, y_train, y_test = air_data_loader.get_data_loader_sklearn()

In [4]:
model_gb = EnsembleModel(name="gradientboosting")
model_rf = EnsembleModel(name="randomforest")

trained_model_gb = model_gb.fit(X_train, y_train, save_checkpoint=False)
trained_model_rf = model_rf.fit(X_train, y_train, save_checkpoint=False)

Creating Gradient Boosting Regressor
Creating Random Forest Regressor
Training model...
Training model...


### Evaluate the trained models

In [5]:
eval_regression_model(trained_model_gb, X_test, y_test)

explained_variance:  0.3407
mean_squared_log_error:  0.6049
R^2:  0.3405466139147818
MAE:  24.257017674855636
MSE:  1279.884552597483
RMSE:  35.775474177115846
MAPE:  95.70604299708907 %


In [6]:
eval_regression_model(trained_model_rf, X_test, y_test)

explained_variance:  0.2974
mean_squared_log_error:  0.6479
R^2:  0.297125354580699
MAE:  25.20959344516901
MSE:  1364.1576797792734
RMSE:  36.93450527324379
MAPE:  101.60010594916862 %


## Pytorch Neural Network

In [7]:
import torch
import torch.nn as nn

from utils.models import NeuralNetwork

In [8]:
train_loader, test_loader = air_data_loader.get_data_loader_pytorch(batch_size=1024)

In [9]:
net = NeuralNetwork(num_input_feat=2)
loss_function = nn.L1Loss() # MAE Loss
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)
num_epochs = 50

In [10]:
for epoch in tqdm(range(num_epochs)):
    current_loss = 0.0
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()

        optimizer.step()

        current_loss += loss.item()
        if i % 10 == 0:
            current_loss = 0.0

# torch.save(net.state_dict(), "trained_models/mlp.pth")

100%|██████████| 50/50 [00:20<00:00,  2.48it/s]
